In [ ]:
import os
import shutil

def populate_train_data(images_dir='../../data/images', train_data_dir='train_data'):
    if not os.path.exists(train_data_dir):
        os.makedirs(train_data_dir)

    for root, dirs, files in os.walk(images_dir):
        rel_path = os.path.relpath(root, images_dir)
        if rel_path == '.':
            rel_path = ''

        target_dir = os.path.join(train_data_dir, rel_path)
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)

        for file in files:
            source_file = os.path.join(root, file)
            target_file = os.path.join(target_dir, file)
            shutil.copy2(source_file, target_file)
            print(f"Copied {source_file} -> {target_file}")

if __name__ == "__main__":
    populate_train_data()


In [ ]:
import os
import csv
import ast

def join_with_and(items):
    """Join list items with commas and 'and' before the last item."""
    if not items:
        return ""
    if len(items) == 1:
        return items[0]
    return ", ".join(items[:-1]) + " and " + items[-1]

train_data_dir = 'train_data'
csv_file = os.path.join(train_data_dir, 'metadata.csv')
image_objects_csv = os.path.join('../../data/output/image_objects.csv')

path_to_label = {}

with open(image_objects_csv, newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        path = row['path'].replace('../../data/images', '').lstrip('/').replace("\\", "/")
        
        try:
            objects = ast.literal_eval(row['objects'])
        except Exception:
            objects = []
        try:
            terrains = ast.literal_eval(row['terrains'])
        except Exception:
            terrains = []
        
        objects_clean  = [obj.strip() for obj in objects if obj.strip()]
        terrains_clean = [tr.strip() for tr in terrains if tr.strip()]
        prompt         = "A top-down fantasy map"
        
        if objects_clean:
            prompt += " showing " + join_with_and(objects_clean)
        
        if terrains_clean:
            if objects_clean:
                prompt += " surrounded by " + join_with_and(terrains_clean) + "."
            else:
                prompt += " surrounded by " + join_with_and(terrains_clean) + "."
        
        if not objects_clean and not terrains_clean:
            prompt += "."
        
        path_to_label[path] = prompt

rows = []
for root, dirs, files in os.walk(train_data_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            rel_dir = os.path.relpath(root, train_data_dir)
            rel_file = os.path.join(rel_dir, file).replace("\\", "/")
            label = path_to_label.get(rel_file, "A top-down fantasy map.")
            rows.append([rel_file, label])

with open(csv_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['file_name', 'label'])
    writer.writerows(rows)

print(f"Metadata CSV saved to: {csv_file}")


In [ ]:
import os
from PIL import Image

def augment_image(image_path):

    img            = Image.open(image_path)
    base_name, ext = os.path.splitext(os.path.basename(image_path))
    folder         = os.path.dirname(image_path)

    augmentations = {
        'rot90' : img.rotate(90, expand=True),
        'rot180': img.rotate(180, expand=True),
        'rot270': img.rotate(270, expand=True),
        'flip_h': img.transpose(Image.FLIP_LEFT_RIGHT),
        'flip_v': img.transpose(Image.FLIP_TOP_BOTTOM)
    }

    for suffix, aug_img in augmentations.items():
        new_filename = f"{base_name}_{suffix}{ext}"
        save_path    = os.path.join(folder, new_filename)
        aug_img.save(save_path)

def process_folder(root_folder):
    for dirpath, _, filenames in os.walk(root_folder):
        for file in filenames:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                full_path = os.path.join(dirpath, file)
                augment_image(full_path)

if __name__ == "__main__":
    image_root = "data/images"
    process_folder(image_root)
